# Retrieval API DEMO

## Code Initialisation
Dependencies and environment initialisation.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `env` file as template.

In [1]:
import os
import json
import requests as r
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
USER_EMAIL = 'retrieval.api@mycompany.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [3]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token OK for user {USERNAME.split('@')[0].split('-')[0]}"))
else:
    display(Markdown(f"**Authentication Failed**: Cannot obtain the Bearer token for the user {USERNAME.split('@')[0].split('-')[0]}"))

**Authentication Successful**: Bearer token OK for user 9ZZZ159100

This creates the headers to be used in the Retrieval API requests.

Additionally, `user_email_hash` is calculated as `user_id`. This must be a unique and constant value per user, so the API statistics keep track of the right number of users. The recommendation is calculating a MD5 hash of the user email. This value is always the same and do not expose any PII.

In the case of `work_id`, this value must be calculated different for each API request. For convenience, it can be taken from the environment's transaction if it exists, or calculated randomly before calling the API. In this case it is calculated randomly before the API request.

In [4]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

user_email_hash = u.md5_hash(USER_EMAIL)
work_id = u.generate_random_guid()

## Factiva Retrieval API Query

### Prompt
Add the prompt to be sent to the Retrieval API

In [5]:
# "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
# "中国2025年计划进行哪些月球探测任务？其主要目标是什么？"   # What are China's planned missions to the Moon in 2025, and what are their primary objectives?
# "今後12か月間の日本の半導体業界の見通しはどのようなものですか?"  # What are the perspectives for the semiconductors industry in Japan for the next 12 months?
# "Summarise the latest earnings report from Microsoft Corp"
# "What are the perspectives of bitcoin for the next 12 months?"
frapi_prompt = "Summarise tariff decisions by the US administration during April 2025 and their impact on the world economy"

Assemble the Retrieval API payload

In [6]:
frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 10,
      "query": {
        "search_filters": [
          {
            "scope": "Language",
            "value": "en"       # en or ja or de or it or es
          }
        ],

        "value": frapi_prompt
      },
      "metrics_data": {
        "user_key": user_email_hash,
        "work_id": work_id,
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}


## Send Retrieval API Request and Receive Chunks Back

In [7]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print Chunks

In [8]:
chunks_list = chunks_resp.json()['data']

# u.print_full_chunks(chunks_list)
u.print_partial_chunks(chunks_list)

### The trade deficit hit a record in March as companies rushed to import goods ahead of tariffs; At some point, tide will shift and imports will 'drop like a stone,' economist says

**MarketWatch** - 2025-05-06 - MRKWC00020250506el56000ul - Lang: en

The numbers: The U.S. international trade deficit, including goods and services, widened 14% in March to a record seasonally adjusted $140.5 billion, ...

---

### Economy Shrinks as Tariffs Take Toll -- WSJ

...

---

### Global stock markets steady amid tariff turbulence; US GDP declines

**Kuwait Times** - 2025-05-04 - MEWKUT0020250505el540002u - Lang: en

KUWAIT: Global markets showed resilience in early May, supported by robust economic data despite lingering tariff uncertainties. In the US, stocks ral...

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example only a few articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is Google Gemini 2.0 Flash. Response generations took a few seconds for a context between 3K to 4K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Gemini Structured Prompt

In [9]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format:
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [10]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration during April 2025 and their impact on the world economy",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.MRKWC00020250506el56000ul",
            "source_name": "MarketWatch",
            "headline": "The trade deficit hit a record in March as companies rushed to import goods ahead of tariffs; At some point, tide will shift and imports will 'drop like a stone,' economist says",
            "publication_date": "2025-05-06",
            "content": "The numbers: The U.S. international trade deficit, including goods and services, widened 14% in March to a record seasonally adjusted $140.5 billion, ..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250501el51000f1",
            "source_name": "Dow Jones Institutional News",
            "headline": "Economy S

### Gemini Request

In [11]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"**Prompt:** {frapi_prompt} \n --------------------"))
display(Markdown(response))

**Prompt:** Summarise tariff decisions by the US administration during April 2025 and their impact on the world economy 
 --------------------

In April 2025, the US administration, under President Trump, implemented and then partially halted a series of tariff decisions that significantly impacted the world economy.

**Tariff Decisions:**

*   **Initial Announcement:** On April 2nd, the US administration announced drastically increased tariffs on virtually all trading partners [5, 4].
*   **Partial Halt:** Implementation of the increased tariffs was later halted for most countries, except for China, with a uniform 10% tariff remaining in place for other trading partners [4].
*   **China Tariffs:** Tariffs on Chinese goods took effect, with China retaliating significantly by placing tariffs on the US [5, 4]. China retaliated in virtually the same way, and placed tariffs on the USA of 125 percent, after the USA increased tariffs on China to 145 percent overall [4].
*   **Broader Strategy:** The tariff announcements were part of a broader strategy to compel foreign governments to reduce trade barriers to American goods, targeting non-tariff barriers like agricultural regulations, taxation systems, currency practices, product standards, legal frameworks, and customs procedures [5]. Countries were given a three-month deadline to make concessions or face new tariffs ranging from 10% to over 50% [5].

**Impact on the World Economy:**

*   **Global Economic Slowdown:** The IMF indicated that the tariff increases and uncertainty would lead to a significant slowdown in global growth in the near term [4]. The IMF downgraded its global growth projection for 2025 to 2.8% and 3% in 2026, down from 3.3% in the January 2025 forecast [8, 4].
*   **US Economic Contraction:** The US economy contracted by 0.3% in the first quarter of 2025, the first contraction since early 2022 [6, 2]. This was partly attributed to businesses rushing to import goods ahead of the tariffs [2]. Net exports subtracted nearly 5 percentage points from headline GDP, the biggest quarterly drag since 1947 [2].
*   **Trade Deficit:** The US international trade deficit widened to a record $140.5 billion in March as companies rushed to import foreign products before tariffs were imposed [7, 1]. Year-to-date, the trade deficit had doubled from the previous year [1].
*   **Uncertainty and Business Impact:** Companies like Ford, Cummins, and Mattel withdrew or suspended their financial outlooks due to economic uncertainty driven by tariffs [3]. Ford estimated tariff-related costs would reduce pretax earnings by $1.5 billion [3]. Tyson Foods saw tariffs and economic uncertainty slicing into Americans' appetite for steaks [3].
*   **Inflation:** President Trump’s tariffs are expected to push inflation higher in the coming months [10].
*   **Impact on Specific Countries:**
    *   **China:** China's factory activity fell as export channels were blocked [6].
    *   **Pakistan:** Pakistan faced a 29% tariff on its exports to the US, potentially impacting its textile industry [9].
    *   **Australia:** The IMF trimmed its forecast for Australian economic growth in 2025 by half a percentage point [8].
*   **Stock Market Volatility:** Financial-market volatility continued due to on-off tariff announcements [2].

**Cited Articles**

- [1] [The trade deficit hit a record in March as companies rushed to import goods ahead of tariffs; At some point, tide will shift and imports will 'drop like a stone,' economist says - MarketWatch - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.MRKWC00020250506el56000ul)
- [2] [Economy Shrinks as Tariffs Take Toll -- WSJ - Dow Jones Institutional News - 2025-05-01](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250501el51000f1)
- [3] [Logistics Report: Tariffs Cloud Forecasts, Drag on Defense; China's Industrial Plan - Dow Jones Institutional News - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250506el56001ae)
- [4] [Tariffs and economic outlook - Pakistan Today - 2025-05-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.ASPTOD0020250505el540002y)
- [5] [Not just tariffs, Donald Trump's trade policies take aim at currency exchange rate, tax, and product standards too - The Times of India - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.TOI0000020250505el56000g1)
- [6] [RECESSION BLUES MARK TRUMP\u2019S 100 TARIFF DAYS - New Indian Express - 2025-05-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.NIEXPR0020250504el540002q)
- [7] [U.S. trade deficit reached historic levels in March due to tariff wars - CE NoticiasFinancieras - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250506el5601wgo)
- [8] [AICD - Underlying inflation back below three per cent - Australian Political and Economic Organizations News via PUBT - 2025-05-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.APEO000020250502el520008d)
- [9] [US tariffs: impact and path forward - Business Recorder - 2025-05-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.AIWBUR0020250504el5400011)
- [10] [Global stock markets steady amid tariff turbulence; US GDP declines - Kuwait Times - 2025-05-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.MEWKUT0020250505el540002u)
